In [1]:
import numpy as np
from math import pi
from copy import deepcopy
from qiskit import *

For a one qubit system we have:
$$\rho = \frac{1}{2}(c_{1}\sigma_{x} + c_{2}\sigma_{y} + c_{3}\sigma_{z} + I)$$

For a two qubit system we have the following equation:
$$\rho = \frac{1}{4}(c_{1}I\otimes\tau_{x} + c_{2}I\otimes\tau_{y} + c_{3}I\otimes\tau_{z} + c_{4}\sigma_{x}\otimes I + c_{5}\sigma_{y}\otimes I + c_{6}\sigma_{z}\otimes I + c_{7}\sigma_{x}\otimes\tau_{x} + c_{8}\sigma_{x}\otimes\tau_{y} + c_{9}\sigma_{x}\otimes\tau_{z} + c_{10}\sigma_{y}\otimes\tau_{x} + c_{11}\sigma_{y}\otimes\tau_{y} + c_{12}\sigma_{y}\otimes\tau_{z} + c_{13}\sigma_{z}\otimes\tau_{x} + c_{14}\sigma_{z}\otimes\tau_{y} + c_{15}\sigma_{z}\otimes\tau_{z} + I\otimes I)$$

where $c_{i}$ are the expectation values.

The following function is used to calculate the expectation value. It takes four arguments including the counts, preference, and shots. 

The reason why this function takes in the dictionary of counts is to avoid measuring twice more for cases like: $$(I\otimes\tau_{x}) ; (\sigma_{x}\otimes I) ; (\sigma_{x}\otimes\tau_{x})$$

The prefence argument is meant to differentiate between expectation value calculations in which an operator is being applied to both qubits (Ex: $\sigma_{z}\otimes\tau_{x}$), to the first qubit (Ex: $\sigma_{x}\otimes I$) and to the second (Ex:$I\otimes\tau_{z}$). It also helps to differentiate one-qubit and two-qubit cases.

In [2]:
#calculate expectation value of observables in a two-qubit system
def expectationValue(counts,pref,shots):
    if pref == 'one':
        expectationValue = 1 * counts.get('0', 0) + -1 * counts.get('1' , 0)
    else:
        if pref == 'b':
            expectationValue = 1 * counts.get('00',0) + 1 * counts.get('11',0) + -1 * counts.get('10', 0) + -1 * counts.get('01',0)
        elif pref == 'f':
            expectationValue = 1 * counts.get('00',0) + -1 * counts.get('11',0) + -1 * counts.get('10', 0) + 1 * counts.get('01',0)
        else:
            expectationValue = 1 * counts.get('00',0) + -1 * counts.get('11',0) + 1 * counts.get('10', 0) + -1 * counts.get('01',0)

    return expectationValue/shots

In [3]:
def getCounts(circuit,q,c,shots):
    job = execute(circuit, backend = Aer.get_backend('qasm_simulator'), shots = shots)
    counts = job.result().get_counts(circuit)
    return counts

The following function calculates the expectation values or the $c_{i}$ coefficients and stores them in a dictionary with appropriate labeling (similar to what the plot_state_paulivec does). Using the expectationValue function above, the densityMatrixComponents function performs 3 measurements for one-qubit systems and is optimized to perform only 9 measurements on 9 circuits for two-qubit systems because the expectationValue function just takes in the dictionary of counts which is already generated inside the densityMatrixComponents, so when used for the cases where there is the Idenitify operator, it does not make any extra measurements.

In [4]:
def densityMatrixComponents(circuit,q,c, shots):
    eValues = {} #Store expectation values
    circuits = [] #store deep-copies of input circuit to perform rotations and measurements
    
    #1-qubit
    if q.size == 1:
        for i in range(3):
            circuits.append(deepcopy(circuit))
        
        circuits[0].ry(-pi/2, q)
        circuits[0].measure(q,c)
        counts0 = getCounts(circuits[0],q,c,shots)
        eValues['X'] = expectationValue(counts0,'one', shots)
        
        circuits[1].rx(pi/2,q)
        circuits[1].measure(q,c)
        counts1 = getCounts(circuits[1],q,c,shots)
        eValues['Y'] = expectationValue(counts1,'one', shots)
        
        circuits[2].measure(q,c)
        counts2 = getCounts(circuits[2],q,c,shots)
        eValues['Z'] = expectationValue(counts2,'one', shots)
        
        eValues['I'] = 1
    #2-qubits
    else:
        for i in range(9):
            circuits.append(deepcopy(circuit))

        circuits[0].ry(-pi/2,1)
        circuits[0].ry(-pi/2,0)
        circuits[0].measure(q,c)
        counts0 = getCounts(circuits[0],q,c,shots)
    #Measureing XI and IX
        eValues['XI'] = expectationValue(counts0,'f', shots)
        eValues['IX'] = expectationValue(counts0,'s', shots)

    #Measure qubit 1 along X and vary qubit 2    
        eValues['XX'] = expectationValue(counts0,'b', shots)

        circuits[1].ry(-pi/2,1)
        circuits[1].rx(pi/2,0)
        circuits[1].measure(q,c)
        counts1 = getCounts(circuits[1],q,c,shots)
        eValues['XY'] = expectationValue(counts1,'b', shots)

        circuits[2].ry(-pi/2,1)
        circuits[2].measure(q,c)
        counts2 = getCounts(circuits[2],q,c,shots)
        eValues['XZ'] = expectationValue(counts2,'b', shots)

    #Measure qubit 1 along Y vary qubit 2
        circuits[3].rx(pi/2,1)
        circuits[3].ry(-pi/2,0)
        circuits[3].measure(q,c)
        counts3 = getCounts(circuits[3],q,c,shots)
        eValues['YX'] =  expectationValue(counts3,'b', shots)

        circuits[4].rx(pi/2,1)
        circuits[4].rx(pi/2,0)
        circuits[4].measure(q,c)
        counts4 = getCounts(circuits[4],q,c,shots)
        eValues['YI'] = expectationValue(counts4,'f', shots)
        eValues['IY'] = expectationValue(counts4,'s', shots)
        eValues['YY'] = expectationValue(counts4,'b', shots)

        circuits[5].rx(pi/2,1)
        circuits[5].measure(q,c)
        counts5 = getCounts(circuits[5],q,c,shots)
        eValues['YZ'] = expectationValue(counts5,'b', shots)

    #Measure qubit 1 along Z vary qubit 2

        circuits[6].ry(-pi/2,0)
        circuits[6].measure(q,c)
        counts6 = getCounts(circuits[6],q,c,shots)
        eValues['ZX'] = expectationValue(counts6,'b', shots)

        circuits[7].rx(pi/2,0)
        circuits[7].measure(q,c)
        counts7 = getCounts(circuits[7],q,c,shots)
        eValues['ZY'] = expectationValue(counts7,'b', shots)

        circuits[8].measure(q,c)
        counts8 = getCounts(circuits[8],q,c,shots)
        eValues['ZI'] = expectationValue(counts8,'f', shots)
        eValues['IZ'] = expectationValue(counts8,'s', shots)
        eValues['ZZ'] = expectationValue(counts8,'b', shots)

        eValues['II'] = 1
        
    for i in eValues:
        eValues[i] = round(eValues[i],2)
    
    return eValues   

In [5]:
#Testing function on Triplet 0 state
q = QuantumRegister(2)
c = ClassicalRegister(2)
circT = QuantumCircuit(q,c)
circT.x(0)  #flip second qubit
circT.ry(pi/2, 1) #superposes first qubit
circT.cx(q[1],q[0])

densityMatrixComponents(circT,q,c, 10000)

{'XI': -0.01,
 'IX': -0.01,
 'XX': 1.0,
 'XY': -0.01,
 'XZ': -0.01,
 'YX': 0.0,
 'YI': 0.01,
 'IY': 0.01,
 'YY': 1.0,
 'YZ': -0.01,
 'ZX': 0.02,
 'ZY': 0.01,
 'ZI': 0.01,
 'IZ': -0.01,
 'ZZ': -1.0,
 'II': 1}

In [6]:
#Testing function for 1 qubit
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = QuantumCircuit(q,c)
circ.h(q)
circ.ry(pi/3,q)
circ.rx(pi,q)

densityMatrixComponents(circ,q,c,10000)

{'X': 0.51, 'Y': -0.01, 'Z': 0.86, 'I': 1}

The following function uses the densityMatrixComponents function to print out the final density matrix for one-qubit and two-qubit systems.

In [7]:
def getDensityMatrix(circuit,q,c):
    components = densityMatrixComponents(circuit,q,c, 10000)
    
    #Pauli operators are stored in a dictionary
    X = np.array([[0.+0.j, 1.+0.j], [1.+0.j, 0.+0.j]])
    Y = np.array([[0.+0.j, 0.-1.j], [0.+1.j, 0.+0.j]])
    Z = np.array([[1.+0.j, 0.+0.j], [0.+0.j, -1.+0.j]])
    I = np.array([[1.+0.j, 0.+0.j], [0.+0.j, 1.+0.j]])
    pauli = {'I': I, 'X': X, 'Y': Y, 'Z': Z}
    
    densityMatrix = 0
    for i,j in pauli.items():
        if q.size == 1:
            densityMatrix += j*components.get(i)
        else:
            for k,l in pauli.items():
                densityMatrix += np.kron(j, l)*components.get(i+k)
            
    print(densityMatrix*(1/2**(q.size)))  

In [8]:
#Simple one-qubit test
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = QuantumCircuit(q,c)
circ.h(q)

getDensityMatrix(circ,q,c)

[[0.5+0.j    0.5+0.005j]
 [0.5-0.005j 0.5+0.j   ]]


In [9]:
#One-qubit test
q = QuantumRegister(1)
c = ClassicalRegister(1)
circ = QuantumCircuit(q,c)
circ.h(q)
circ.ry(pi/3,q)
circ.rx(pi,q)

getDensityMatrix(circ,q,c)

[[0.935+0.j    0.25 +0.005j]
 [0.25 -0.005j 0.065+0.j   ]]


In [10]:
#Test Singlet State
q = QuantumRegister(2)
c = ClassicalRegister(2)
circS = QuantumCircuit(q,c)
circS.x(0)  #flip second qubit
circS.ry(-pi/2, 1) #superposes first qubit
circS.cx(q[1],q[0])

getDensityMatrix(circS,q,c)         

[[ 0.    +0.j      0.0025+0.0025j  0.01  -0.0025j  0.    +0.005j ]
 [ 0.0025-0.0025j  0.505 +0.j     -0.5   +0.j     -0.005 -0.0075j]
 [ 0.01  +0.0025j -0.5   +0.j      0.495 +0.j     -0.0075+0.0075j]
 [ 0.    -0.005j  -0.005 +0.0075j -0.0075-0.0075j  0.    +0.j    ]]


In [11]:
#Test Triplet State
q = QuantumRegister(2)
c = ClassicalRegister(2)
circT = QuantumCircuit(q,c)
circT.x(0)  #flip second qubit
circT.ry(pi/2, 1) #superposes first qubit
circT.cx(q[1],q[0])


getDensityMatrix(circT,q,c) 

[[0.   +0.j     0.   +0.j     0.   +0.0025j 0.   +0.0025j]
 [0.   +0.j     0.495+0.j     0.5  -0.0025j 0.   -0.0025j]
 [0.   -0.0025j 0.5  +0.0025j 0.505+0.j     0.   +0.j    ]
 [0.   -0.0025j 0.   +0.0025j 0.   +0.j     0.   +0.j    ]]
